<style type="text/css">
.tf  {border-collapse:collapse;border-spacing:0;width:100%}
.tf td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tf th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tf .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.col1 { width: 20%;}
.col2 { width: 80%;}
</style>
<table class="tf">
<thead>
  <tr>
    <th class="tg-0pky col1">Name</th>
    <th class="tg-0pky col2">Hatim Sawai</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky col1">UID No.</td>
    <td class="tg-0pky col2">2021300108</td>
  </tr>
  <tr>
    <td class="tg-0pky col1">Experiment No.</td>
    <td class="tg-0pky col2">4</td>
  </tr>
</tbody>
</table>

## <p style="text-align:center;font-weight:500;">Experiment 4</p>
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;width:100%}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 10px;word-break:normal;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top;}
.col1 { width: 20%;}
.col2 { width: 80%;}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0pky col1">Aim</th>
    <th class="tg-0pky col2">Classification using suitable classification model (NB).
    </th>
  </tr>
</thead>
</table> 

### 1. Installation of NLTK and downloading the required corpus

In [45]:
import nltk
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import wordnet
from prettytable import PrettyTable
from nltk.tokenize import word_tokenize
from pattern.text.en import pluralize, conjugate, comparative
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer, SnowballStemmer

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shabb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\shabb\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shabb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

<p style="page-break-after:always;"></p>

### 2. Load Dataset & Preprocessing

In [46]:
# load csv dataset
def load_dataset(csv_path):
    df = pd.read_csv(csv_path)
    documents = df['Text'].tolist()
    labels = df['Label'].tolist()
    return documents, labels

In [47]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = text.replace("\n", " ") # remove \n
    text = re.sub(r'\W', ' ', text)  # Remove non-word characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespaces
    return text

In [48]:
# build vocabulary
def build_vocabulary(documents):
    vocabulary = set()
    for document in documents:
        words = document.split()
        vocabulary.update(words)
    return list(vocabulary)

In [49]:
def count_word_occurrences(document, vocabulary):
    word_counts = {word: 0 for word in vocabulary}
    words = document.split()
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
    return word_counts

<p style="page-break-after:always;"></p>

### 3. Making the model

In [50]:
def train_naive_bayes(documents, labels):
    unique_labels = set(labels)
    label_probs = {label: labels.count(label) / len(labels) for label in unique_labels}
    
    vocabulary = build_vocabulary(documents)
    
    word_probs = {label: {word: 0 for word in vocabulary} for label in unique_labels}
    
    for i, document in enumerate(documents):
        label = labels[i]
        word_counts = count_word_occurrences(document, vocabulary)
        for word, count in word_counts.items():
            word_probs[label][word] += count
    
    for label in unique_labels:
        total_count = sum(word_probs[label].values())
        for word, count in word_probs[label].items():
            word_probs[label][word] = (count + 1) / (total_count + len(vocabulary))
    
    return label_probs, word_probs, vocabulary

In [51]:
def classify_naive_bayes(document, label_probs, word_probs, vocabulary):
    document = preprocess(document)
    word_counts = count_word_occurrences(document, vocabulary)
    scores = {label: np.log(label_probs[label]) for label in label_probs}
    for label in label_probs:
        for word, count in word_counts.items():
            if word in vocabulary:
                scores[label] += count * np.log(word_probs[label][word])
    return max(scores, key=scores.get)

In [63]:
def evaluate_classifier(test_documents, test_labels, label_probs, word_probs, vocabulary):
    true_positive, false_positive, true_negative, false_negative = 0, 0, 0, 0
    print("Evaluating the classifier...")
    for i, document in enumerate(test_documents):
        predicted_label = classify_naive_bayes(document, label_probs, word_probs, vocabulary)
        true_label = test_labels[i]
        
        if predicted_label == true_label:
            if predicted_label == 1:  # Sports
                true_positive += 1
            else:
                true_negative += 1
        else:
            if predicted_label == 1:  # Sports
                false_positive += 1
            else:
                false_negative += 1
        print(f"Document {i+1} - True Label: {true_label}, Predicted Label: {predicted_label}")
    
    accuracy = (true_positive + true_negative) / len(test_documents)
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    confusion_matrix = np.array([[true_positive, false_positive], [false_negative, true_negative]])
    
    return accuracy, precision, recall, f1_score, confusion_matrix

<p style="page-break-after:always;"></p>

### 4. Model Evaluation

In [53]:
# Load the dataset from CSV
csv_path = "./Dataset/df_file.csv"
documents, labels = load_dataset(csv_path)
print(f"Loaded {len(documents)} documents")

Loaded 2225 documents


In [58]:
# Split the dataset into training and testing sets (80% training, 20% testing) using sklearn
from sklearn.model_selection import train_test_split
train_documents, test_documents, train_labels, test_labels = train_test_split(documents, labels, test_size=0.2, random_state=42)

print(f"Training set: {len(train_documents)} documents")

Training set: 1780 documents


In [59]:
# Train the Naive Bayes classifier
label_probs, word_probs, vocabulary = train_naive_bayes(train_documents, train_labels)

In [64]:
# Evaluate the classifier
accuracy, precision, recall, f1_score, confusion_matrix = evaluate_classifier(test_documents, test_labels, label_probs, word_probs, vocabulary)

Evaluating the classifier...


KeyboardInterrupt: 

<p style="page-break-after:always;"></p>

### 5. Results

In [ ]:
# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("\nConfusion Matrix:")
print(confusion_matrix)

<p style="page-break-after:always;"></p>

### 6. Curiosity Questions
<p style="font-weight:500;">Q1. Some Terminologies</p>  
 
1. **Corpus**: A corpus is a collection of written texts and is used for language research and development.  
2. **Bigram**: A bigram is a sequence of two adjacent elements from a string of tokens, which are typically letters, syllables, or words. A bigram is an n-gram for n=2.  
3. **Add-one Smoothing**: Add-one smoothing is a simple method to smooth zero probabilities in a probability distribution. It is also known as Laplace smoothing.  
4. **Sparse Table**: A sparse table is a table that has a large number of cells with zero values. It is a table in which most of the entries are zero.  
5. **Word Form:** the inflected form as it actually appears in the corpus.  
6. **Lemma:** an abstract form, shared by word forms having the same stem, part of speech, and word sense – stands for the class of words with stem.  
7. **Types:** number of distinct words in a corpus (vocabulary size).  
8. **Tokens:** total number of words in a corpus.  

<p style="font-weight:500;">Q2. What is Bi-gram Model? How to calculate the probability of a sentence?</p>
Ans: A bigram model is a type of n-gram language model where "n" equals 2. In simpler terms, it predicts the next word in a sentence based solely on the word that came before it. This assumption, termed the Markov assumption, states that the probability of a word depends only on the previous word, ignoring all preceding context. While simplistic, bigram models hold historical significance as the foundation for more complex language models.  

**Calculating Sentence Probability:**  
Here's how to calculate the probability of a sentence in a bigram model:  
1. Tokenize the sentence: Divide the sentence into individual words (tokens).
2. Estimate bigram probabilities: Calculate the probability of each word following the previous word in the training corpus. This uses the formula: **P(w_i | w_(i-1)) = Count(w_(i-1), w_i) / Count(w_(i-1))**    

where:  
P(w_i | w_(i-1)) is the probability of word w_i given the previous word w_(i-1).  
Count(w_(i-1), w_i) is the number of times the bigram (w_(i-1), w_i) appears in the training corpus.  
Count(w_(i-1)) is the number of times the word w_(i-1) appears in the training corpus.  
Multiply probabilities: Multiply the individual bigram probabilities for each word pair in the sentence.  

3. Smoothing: Since most word pairs might not be present in the corpus, apply smoothing techniques like Laplace smoothing or Witten-Bell discounting to adjust probabilities and avoid zero probabilities.  

### 6. Conclusion
In this experiment we learned about bigrams and add-one smoothing. We calculated bigrams from a given corpus and displayed the bigram probability table. We also calculated the probability of a sentence using bigram probabilities. We also applied add-one smoothing on the sparse bigram table.